In [1]:
# Topgyal Gurung
# Decision Tree Algorithm from scratch on monk dataset
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
# load and prepare dataset
# train_set
monk1_train=pd.read_csv("~/train1.csv")
monk2_train=pd.read_csv("~/train2.csv")
monk3_train=pd.read_csv("~/train3.csv")
# test_set
monk1_test=pd.read_csv("~/test1.csv")
monk2_test=pd.read_csv("~/test2.csv")
monk3_test=pd.read_csv("~/test3.csv")

# monk3_test=pd.read_csv("/Users/topgyalgurung/Desktop/monk-dataset/test3.csv", header=None)

In [3]:
# drop Id 
monk1_train=monk1_train.drop("Id", axis=1)
monk2_train=monk2_train.drop("Id",axis=1)
monk3_train=monk3_train.drop("Id",axis=1)

monk1_test=monk1_test.drop("Id",axis=1)
monk2_test=monk2_test.drop("Id",axis=1)
monk3_test=monk3_test.drop("Id",axis=1)
# monk3_test=monk3_test.drop([7],axis=1)

In [4]:
monk1_test.head(2)

,class,a1,a2,a3,a4,a5,a6
0,1,1,1,1,1,1,1
1,1,1,1,1,1,1,2


In [5]:
# dataframe into numpy array
monk1_data=monk1_train.values
monk2_data=monk2_train.values
monk3_data=monk3_train.values

In [6]:
# pandas dataframe
monk1_train.head(2) # print first 2 rows

,class,a1,a2,a3,a4,a5,a6
0,1,1,1,1,1,3,1
1,1,1,1,1,1,3,2


In [7]:
# numpy 2d array
monk1_data[:2]  # print first 2 rows

array([[1, 1, 1, 1, 1, 3, 1],
       [1, 1, 1, 1, 1, 3, 2]])

In [8]:
monk1_data[:,0] # print first class data (first column)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## HELPER FUNCTIONS

In [9]:
label_column=monk1_data[:,0]   # first column class data
np.unique(label_column,return_counts=True) # 0,1

(array([0, 1]), array([62, 62]))

In [10]:
unique_classes,counts_unique_classes=np.unique(label_column,return_counts=True)
_,counts_unique_classes=np.unique(label_column,return_counts=True)

In [11]:
print(unique_classes) 
print(len(unique_classes))
print(counts_unique_classes) # 50-50 

[0 1]
2
[62 62]


In [12]:
# base case 
def check_purity(data):
    label_column=data[:,0]
    unique_classes=np.unique(label_column)
    
    if len(unique_classes)==1:
        return True
    else:
        return False

In [13]:
check_purity(monk1_data) # check if monk1 pure( only one class)

False

In [14]:
def classify_data(data):
    label_column=data[:,0]
    unique_classes,counts_unique_classes=np.unique(label_column,return_counts=True) # [0 1] , [62 62]
    index=counts_unique_classes.argmax()  # use max class as index 
    classification=unique_classes[index]
    return classification

### POTENTIAL SPLITS

In [15]:
monk1_data.shape  # 124 rows, 7 columns

(124, 7)

In [16]:
# test with monk1_data 
_,n_columns=monk1_train.shape
for column_i in range(1,n_columns):  #iterate over column (1,7)
    values=monk1_data[:,column_i]
    unique_values,unique_value_counts=np.unique(values,return_counts=True)
    if column_i!=0:
        print("a-",column_i,"values=", unique_values) 
        print("counts=", unique_value_counts)
        print("")

a- 1 values= [1 2 3]
counts= [45 42 37]

a- 2 values= [1 2 3]
counts= [35 42 47]

a- 3 values= [1 2]
counts= [65 59]

a- 4 values= [1 2 3]
counts= [42 39 43]

a- 5 values= [1 2 3 4]
counts= [29 31 30 34]

a- 6 values= [1 2]
counts= [56 68]



In [17]:
def get_potential_splits(data):
    potential_splits={}
    _,n_columns=data.shape  #iterate over column
    for column_i in range(1,n_columns):  # column index exclude class
        values = data[:, column_i]
        unique_values = np.unique(values)
        potential_splits[column_i]=unique_values
    return potential_splits  

In [18]:
get_potential_splits(monk1_data)

{1: array([1, 2, 3]),
 2: array([1, 2, 3]),
 3: array([1, 2]),
 4: array([1, 2, 3]),
 5: array([1, 2, 3, 4]),
 6: array([1, 2])}

# SPLIT DATA

In [19]:
# split data to data equal if equal and data_not for not equal
def split_data(data,split_column,split_value):
    #random.shuffle(data)
    split_column_values=data[:,split_column]
    true_data=data[split_column_values==split_value]
    false_data=data[split_column_values!=split_value]
    return true_data,false_data  

In [20]:
# monk1 split data a1==1?
true_data,false_data=split_data(monk1_data,1,1)

In [21]:
len(true_data) +len(false_data)  # should be 124 

124

In [22]:
true_data[:4]

array([[1, 1, 1, 1, 1, 3, 1],
       [1, 1, 1, 1, 1, 3, 2],
       [1, 1, 1, 1, 3, 2, 1],
       [1, 1, 1, 1, 3, 3, 2]])

In [23]:
false_data[:4]

array([[0, 2, 1, 1, 1, 3, 1],
       [0, 2, 1, 1, 1, 3, 2],
       [1, 2, 1, 1, 2, 1, 1],
       [1, 2, 1, 1, 2, 1, 2]])

# entropy 

In [24]:
label_column=monk1_data[:,0] # first column=class

In [25]:
_,counts=np.unique(label_column,return_counts=True)
print(counts)
counts.sum()
counts/counts.sum()

[62 62]


array([0.5, 0.5])

In [26]:
# entropy: measure of uncertainty

def entropy(data):
    label_column=data[:,0]
    _,counts=np.unique(label_column,return_counts=True)
    
    probability=counts/counts.sum()
    entropy=sum(probability*-np.log2(probability))
    return entropy

In [27]:
entropy(monk1_data)

1.0

In [28]:
entropy(monk2_data)

0.957117428264771

In [29]:
entropy(monk3_data)

0.999806132804711

In [30]:
def overall_entropy(true_data, false_data):
    n=len(true_data)+len(false_data)
    p_true_data=len(true_data)/n
    p_false_data=len(false_data)/n
    overall_entropy=(p_true_data * entropy(true_data) + p_false_data * entropy(false_data))
    return overall_entropy

In [31]:
overall_entropy(true_data,false_data)

0.9402478084953086

In [32]:
# information gain (S)= entropy(S)- weighted entropy
# weighted entropy=[(weighted average * entropy(each feature))]
# split_attr takes name of attribute to calculate infogain for

# def info_gain(dat_below, data_above, uncertainty):
#     p = float(len(true_data)) /(len(true_data)+len(false_data))
#     return uncertainty - p * entropy(true_data) -(1 - p) * entropy(false_data)

In [33]:
len(monk1_data)

124

In [34]:
def info_gain(data, true_data, false_data):
    p=len(true_data)/len(data)
    info_gain=entropy(data) - p* entropy(true_data)-(1-p) *entropy(false_data)
    return info_gain

In [35]:
# test 
true_data,false_data=split_data(monk1_data,1,1) # a1=1
info_gain(monk1_data,true_data,false_data)

0.059752191504691354

In [36]:
true_data,false_data=split_data(monk1_data,1,2) # a1=2
info_gain(monk1_data,true_data,false_data)

0.0008380397143500362

In [37]:
true_data,false_data=split_data(monk1_data,1,3) #a1=3
info_gain(monk1_data,true_data,false_data)

0.05153479275836825

In [38]:
true_data,false_data=split_data(monk1_data,2,1) #a2=1
info_gain(monk1_data,true_data,false_data)

0.005804416680231128

In [39]:
true_data,false_data=split_data(monk1_data,2,2) #a2=2
info_gain(monk1_data,true_data,false_data)

0.0008380397143500362

In [40]:
true_data,false_data=split_data(monk1_data,2,3) #a2=3
info_gain(monk1_data,true_data,false_data)

0.0017948307725639268

# Determine best split

In [41]:
get_potential_splits(monk1_data)

{1: array([1, 2, 3]),
 2: array([1, 2, 3]),
 3: array([1, 2]),
 4: array([1, 2, 3]),
 5: array([1, 2, 3, 4]),
 6: array([1, 2])}

In [42]:
# best_split
def best_split(data,potential_splits):
    temp_entropy=999  #arbitary large num
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            true_data,false_data=split_data(data,split_column=column_index,split_value=value)
            #info_gain
            current_overall_entropy=overall_entropy(true_data,false_data)
            #print(current_overall_entropy)
            if current_overall_entropy<=temp_entropy:
                temp_entropy=current_overall_entropy
                best_split_column=column_index
                best_split_value=value
    return best_split_column, best_split_value

In [43]:
potential_splits=get_potential_splits(monk1_data)

In [44]:
print("a","val","curr_over_entropy")
for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below,data_above=split_data(monk1_data,split_column=column_index,split_value=value)
            current_overall_entropy=overall_entropy(data_below,data_above)
            print(column_index," ", value," ", current_overall_entropy)

a val curr_over_entropy
1   1   0.9402478084953086
1   2   0.9991619602856499
1   3   0.9484652072416317
2   1   0.9941955833197689
2   2   0.9991619602856499
2   3   0.9982051692274361
3   1   0.9952924333827027
3   2   0.9952924333827027
4   1   0.9789033341385207
4   2   0.9822543194854161
4   3   0.9997928807470515
5   1   0.7137992387792323
5   2   0.9795042028631318
5   3   0.9834727618772467
5   4   0.9654876088845705
6   1   0.999242144284136
6   2   0.999242144284136


# representation of decision tree
sub_tree={question:[1,2,3,4]} # depends on attribute

In [45]:
potential_splits=get_potential_splits(monk1_data)

In [46]:
best_split(monk1_data,potential_splits) # a5 on node

(5, 1)

In [83]:
best_split(monk2_data,potential_splits) #a4 supposed to be a5

(4, 1)

In [84]:
best_split(monk3_data,potential_splits)  # a2 

(2, 3)

In [47]:
potential_splits=get_potential_splits(monk1_data)
split_column,split_value=best_split(monk1_data,potential_splits)
true_data,false_data=split_data(monk1_data,split_column,split_value)

In [48]:
# recursive decision_tree algorithm
def decision_tree(train_set,counter=0):
    if counter==0:
        global column_headers
        column_headers=train_set.columns
        data=train_set.values  # numpy array
    else:
        data= train_set   # pandas dataframe
    
    # base case
    if check_purity(data):
        classification=classify_data(data)
        return classification
    # recursive
    else:
        counter+=1
        # run helper functions
        potential_splits=get_potential_splits(data)
        split_column,split_value=best_split(data,potential_splits)
        true_data,false_data=split_data(data,split_column,split_value)

        # sub_tree
        feature_name=column_headers[split_column]
        question="{} == {}".format(feature_name,split_value)
        sub_tree={question:[]} #empty list
        # find ans (recursive part)
        yes_ans=decision_tree(true_data,counter)
        no_ans=decision_tree(false_data,counter)
        
        if yes_ans==no_ans:
            sub_tree=yes_ans
        else:
            sub_tree[question].append(yes_ans)
            sub_tree[question].append(no_ans)

        return sub_tree

In [49]:
tree=decision_tree(monk1_train)

In [50]:
tree2=decision_tree(monk2_train)

In [51]:
tree3=decision_tree(monk3_train)

In [52]:
pprint(tree)

{'a5 == 1': [1,
             {'a1 == 1': [{'a2 == 1': [1, 0]},
                          {'a2 == 1': [0,
                                       {'a5 == 3': [{'a2 == 3': [{'a1 == 3': [1,
                                                                              0]},
                                                                 {'a1 == 3': [0,
                                                                              1]}]},
                                                    {'a4 == 1': [{'a6 == 2': [{'a3 == 2': [1,
                                                                                           {'a5 == 4': [{'a2 == 3': [1,
                                                                                                                     0]},
                                                                                                        1]}]},
                                                                              1]},
                                   

In [53]:
pprint(tree2)

{'a4 == 1': [{'a5 == 1': [{'a6 == 2': [{'a3 == 2': [{'a2 == 3': [1, 0]}, 0]},
                                       0]},
                          {'a1 == 1': [{'a6 == 2': [{'a3 == 2': [1, 0]}, 0]},
                                       {'a6 == 2': [{'a3 == 2': [{'a2 == 1': [1,
                                                                              0]},
                                                                 {'a2 == 1': [0,
                                                                              1]}]},
                                                    {'a3 == 2': [{'a2 == 1': [0,
                                                                              1]},
                                                                 0]}]}]}]},
             {'a5 == 2': [{'a3 == 2': [{'a2 == 2': [0,
                                                    {'a1 == 1': [1,
                                                                 {'a2 == 3': [0,
                          

In [54]:
pprint(tree3)

{'a2 == 3': [{'a4 == 1': [{'a3 == 2': [0, {'a1 == 1': [0, 1]}]}, 0]},
             {'a5 == 4': [0,
                          {'a5 == 3': [{'a3 == 2': [1,
                                                    {'a1 == 3': [1,
                                                                 {'a6 == 2': [{'a4 == 3': [{'a2 == 2': [{'a1 == 2': [0,
                                                                                                                     1]},
                                                                                                        1]},
                                                                                           1]},
                                                                              {'a4 == 1': [{'a2 == 2': [1,
                                                                                                        0]},
                                                                                           0]}]}]}]},
             

In [55]:
# give accuracy for each tree

## classification

In [56]:
# just use row first for eg
example=monk1_test.iloc[0] # first row
example

class    1
a1       1
a2       1
a3       1
a4       1
a5       1
a6       1
Name: 0, dtype: int64

In [57]:
example2=monk1_test.iloc[51] 
example2

class    0
a1       1
a2       2
a3       1
a4       1
a5       2
a6       2
Name: 51, dtype: int64

In [58]:
# list
tree.keys()

dict_keys(['a5 == 1'])

In [59]:
# get string
list(tree.keys())[0]

'a5 == 1'

In [60]:
question=list(tree.keys())[0]
feature_name,operator,value=question.split(" ")
feature_name

'a5'

In [61]:
def classify_test(example,tree):
    question=list(tree.keys())[0]
    feature_name,operator,value=question.split(" ")
    
    if str(example[feature_name])== value:
        ans=tree[question][0]
    else:
        ans=tree[question][1]
    # base case
    if not isinstance(ans,dict):
        return ans
    
    # recursion
    else:
        residual_tree=ans
        return classify_test(example,residual_tree)

In [62]:
# def prune(tree):

In [63]:
classify_test(example,tree)

1

In [64]:
classify_test(example2,tree)

0

In [65]:
classify_test(monk1_train,tree)

1

In [66]:
classify_test(monk1_test,tree)

1

In [67]:
classify_test(monk2_train,tree2)

1

In [68]:
classify_test(monk2_test,tree2)

1

In [69]:
classify_test(monk2_test,tree3)

1

In [70]:
classify_test(monk3_test,tree3)

1

## Accuracy

In [71]:
monk1_train.apply(classify_test,args=(tree,),axis=1)

0      1
1      1
2      1
3      1
4      1
      ..
119    1
120    1
121    1
122    1
123    1
Length: 124, dtype: int64

In [72]:
monk1_train["classification"]=monk1_train.apply(classify_test,args=(tree,),axis=1)

In [73]:
monk1_train["classification"]==monk1_train["class"]

0      True
1      True
2      True
3      True
4      True
       ... 
119    True
120    True
121    True
122    True
123    True
Length: 124, dtype: bool

In [74]:
def accuracy(df,tree):
    df["classification"]=df.apply(classify_test, args=(tree,),axis=1)
    df["classification_correct"]=df["classification"]==df["class"]
    
    accuracy=df["classification_correct"].mean()
    return accuracy

In [75]:
accuracy(monk1_train,tree)

1.0

In [76]:
accuracy(monk1_test,tree)

0.9004629629629629

In [77]:
accuracy(monk2_train,tree2)

1.0

In [78]:
accuracy(monk2_test,tree2)

0.8703703703703703

In [79]:
accuracy(monk3_train,tree3)

1.0

In [80]:
accuracy(monk3_test,tree3)

0.9004629629629629

# decision tree

In [85]:

# 1. build tree
# 2. check performance on train again 
# 3. post pruning 
# 4. calculate error 
# 5. use tree to classify test
# 6. calculate error 


In [82]:
# import matplotlib.pyplot as plt
# plt.plot(accuracy, depth, marker='o',label="accuracy")
# plt.title('MONK-1 Classify vs Accuracy')
# plt.xlabel('Depth')
# plt.ylabel('Accuracy')
# plt.legend(loc='upper right',frameon=False)
# plt.show()